# 第9章: 事前学習済み言語モデル（BERT型）

本章では、BERT型の事前学習済みモデルを利用して、マスク単語の予測や文ベクトルの計算、評判分析器（ポジネガ分類器）の構築に取り組む。

## 80. トークン化

"The movie was full of incomprehensibilities."という文をトークンに分解し、トークン列を表示せよ。

In [1]:
from transformers import AutoTokenizer

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

sentence = "The movie was full of incomprehensibilities."
tokens = tokenizer.tokenize(sentence)
print(tokens)

['the', 'movie', 'was', 'full', 'of', 'inc', '##omp', '##re', '##hen', '##si', '##bilities', '.']


## 81. マスクの予測

"The movie was full of [MASK]."の"[MASK]"を埋めるのに最も適切なトークンを求めよ。

In [2]:
from transformers import pipeline

model_name = "google-bert/bert-base-uncased"
fill_mask = pipeline(
    "fill-mask",
    model=model_name,
)

masked_sentence = "The movie was full of [MASK]."
prediction = fill_mask(masked_sentence)

print(f"[MASK]: {prediction[0]['token_str']}")

2025-06-12 00:18:55.693974: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 00:18:55.725228: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-12 00:18:55.725256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-12 00:18:55.726044: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-12 00:18:55.731063: I tensorflow/core/platform/cpu_feature_guar

[MASK]: fun


## 82. マスクのtop-k予測

"The movie was full of [MASK]."の"[MASK]"に埋めるのに適切なトークン上位10個と、その確率（尤度）を求めよ。

In [3]:
from transformers import pipeline

model_name = "google-bert/bert-base-uncased"
fill_mask = pipeline(
    "fill-mask",
    model=model_name,
)

masked_sentence = "The movie was full of [MASK]."
prediction = fill_mask(masked_sentence, top_k=10)

for i in range(10):
    print(
        f"[MASK]: {prediction[i]['token_str']} (Score: {prediction[i]['score']:.4f})"
    )

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[MASK]: fun (Score: 0.1071)
[MASK]: surprises (Score: 0.0663)
[MASK]: drama (Score: 0.0447)
[MASK]: stars (Score: 0.0272)
[MASK]: laughs (Score: 0.0254)
[MASK]: action (Score: 0.0195)
[MASK]: excitement (Score: 0.0190)
[MASK]: people (Score: 0.0183)
[MASK]: tension (Score: 0.0150)
[MASK]: music (Score: 0.0146)


## 83. CLSトークンによる文ベクトル

以下の文の全ての組み合わせに対して、最終層の[CLS]トークンの埋め込みベクトルを用いてコサイン類似度を求めよ。

- "The movie was full of fun."
- "The movie was full of excitement."
- "The movie was full of crap."
- "The movie was full of rubbish."


In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

sentences = [
    "The movie was full of fun.",
    "The movie was full of excitement.",
    "The movie was full of crap.",
    "The movie was full of rubbish.",
]

# トークナイザーを使って文をトークン化し、テンソルに変換
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# 文の埋め込みを取得
with torch.no_grad():
    outputs = model(**inputs)
    
# CLSトークンの埋め込みを取得
cls_embeddings = outputs.last_hidden_state[:, 0, :].numpy()

n = len(sentences)
similarity_matrix = np.zeros((n, n))

# コサイン類似度を計算
for i in range(n):
    for j in range(n):
        similarity_matrix[i, j] = cosine_similarity([cls_embeddings[i]], [cls_embeddings[j]])[0][0]

for i in range(n):
    for j in range(i+1, n):
        print(f" \"{sentences[i]}\"  \"{sentences[j]}\"  ==> {similarity_matrix[i, j]:.4f}")

 "The movie was full of fun."  "The movie was full of excitement."  ==> 0.9881
 "The movie was full of fun."  "The movie was full of crap."  ==> 0.9558
 "The movie was full of fun."  "The movie was full of rubbish."  ==> 0.9475
 "The movie was full of excitement."  "The movie was full of crap."  ==> 0.9541
 "The movie was full of excitement."  "The movie was full of rubbish."  ==> 0.9487
 "The movie was full of crap."  "The movie was full of rubbish."  ==> 0.9807


## 84. 平均による文ベクトル

以下の文の全ての組み合わせに対して、最終層の埋め込みベクトルの平均を用いてコサイン類似度を求めよ。

- "The movie was full of fun."
- "The movie was full of excitement."
- "The movie was full of crap."
- "The movie was full of rubbish."

In [5]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

sentences = [
    "The movie was full of fun.",
    "The movie was full of excitement.",
    "The movie was full of crap.",
    "The movie was full of rubbish.",
]

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

input_mask_expanded = inputs['attention_mask'].unsqueeze(-1).expand(last_hidden_states.size()).float()
sum_embeddings = torch.sum(last_hidden_states * input_mask_expanded, 1)
sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
mean_embeddings = sum_embeddings / sum_mask
mean_embeddings = mean_embeddings.numpy()

n = len(sentences)
similarity_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        similarity_matrix[i, j] = cosine_similarity(
            [mean_embeddings[i]], [mean_embeddings[j]]
        )[0][0]

for i in range(n):
    for j in range(i + 1, n):
        print(f' "{sentences[i]}"  "{sentences[j]}"  ==> {similarity_matrix[i, j]:.4f}')

 "The movie was full of fun."  "The movie was full of excitement."  ==> 0.9568
 "The movie was full of fun."  "The movie was full of crap."  ==> 0.8490
 "The movie was full of fun."  "The movie was full of rubbish."  ==> 0.8169
 "The movie was full of excitement."  "The movie was full of crap."  ==> 0.8352
 "The movie was full of excitement."  "The movie was full of rubbish."  ==> 0.7938
 "The movie was full of crap."  "The movie was full of rubbish."  ==> 0.9226


## 85. データセットの準備

[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されている[Stanford Sentiment Treebank (SST)](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip) から訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、さらに全てのテキストはトークン列に変換せよ。

In [6]:
import pandas as pd
import torch
from transformers import AutoTokenizer
from datasets import load_dataset

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset = load_dataset("glue", "sst2", split="train")
dev_dataset = load_dataset("glue", "sst2", split="validation")

def pre_process(examples):
    tokenized = tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokenized["labels"] = examples["label"]
    return tokenized

encoded_train_dataset = train_dataset.map(pre_process, remove_columns=train_dataset.column_names)
encoded_dev_dataset = dev_dataset.map(pre_process, remove_columns=dev_dataset.column_names)



Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [7]:
print(train_dataset[0])
print(encoded_train_dataset[0])

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}
{'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## 86. ミニバッチの作成

85で読み込んだ訓練データの一部（例えば冒頭の4事例）に対して、パディングなどの処理を行い、トークン列の長さを揃えてミニバッチを構成せよ。

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
batch = data_collator(encoded_train_dataset[:2])
print(batch)

{'input_ids': tensor([[  101,  5342,  2047,  3595,  8496,  2013,  1996, 18643,  3197,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

## 87. ファインチューニング

訓練セットを用い、事前学習済みモデルを極性分析タスク向けにファインチューニングせよ。検証セット上でファインチューニングされたモデルの正解率を計測せよ。

In [12]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import (
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)


class_label = train_dataset.features["label"]

model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=class_label.num_classes,
    label2id = {label: id for id, label in enumerate(class_label.names)},
    id2label = {id: label for id, label in enumerate(class_label.names)},
)

train_args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs= 3,
    weight_decay=0.01,
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_dev_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")
    
torch.save(model.state_dict(), "./model/No_87.pt")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/kawata/.local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_4108591/3841690126.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.181000,0.241833,0.919725,0.917411,0.925676,0.921525
2,0.128600,0.273656,0.917431,0.890756,0.954955,0.921739
3,0.092200,0.288256,0.920872,0.908497,0.939189,0.923588


Evaluation results: {'eval_loss': 0.288255900144577, 'eval_accuracy': 0.9208715596330275, 'eval_precision': 0.9084967320261438, 'eval_recall': 0.9391891891891891, 'eval_f1': 0.9235880398671097, 'eval_runtime': 2.658, 'eval_samples_per_second': 328.07, 'eval_steps_per_second': 10.534, 'epoch': 3.0}


## 88. 極性分析

問題87でファインチューニングされたモデルを用いて、以下の文の極性を予測せよ。

- "The movie was full of incomprehensibilities."
- "The movie was full of fun."
- "The movie was full of excitement."
- "The movie was full of crap."
- "The movie was full of rubbish."


In [13]:
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "negative", 1: "positive"}
)

model.load_state_dict(torch.load("./model/No_87.pt"))
model.to(device)
model.eval()

sentences = [
    "The movie was full of incomprehensibilities.",
    "The movie was full of fun.",
    "The movie was full of excitement.",
    "The movie was full of crap.",
    "The movie was full of rubbish."
]

with torch.no_grad():
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        prediction = torch.argmax(logits, dim=-1).item()
        
        label = model.config.id2label[prediction]
        confidence = probabilities[0][prediction].item()
        print(f"文: '{sentence}'")
        print(f"予測: {label} (確信度: {confidence:.4f})")
        print("-" * 50)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_4108591/722172470.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_g

文: 'The movie was full of incomprehensibilities.'
予測: negative (確信度: 0.9979)
--------------------------------------------------
文: 'The movie was full of fun.'
予測: positive (確信度: 0.9993)
--------------------------------------------------
文: 'The movie was full of excitement.'
予測: positive (確信度: 0.9991)
--------------------------------------------------
文: 'The movie was full of crap.'
予測: negative (確信度: 0.9983)
--------------------------------------------------
文: 'The movie was full of rubbish.'
予測: negative (確信度: 0.9985)
--------------------------------------------------


## 89. アーキテクチャの変更

問題87とは異なるアーキテクチャ（例えば[CLS]トークンを用いるか、各トークンの最大値プーリングを用いるなど）の分類モデルを設計し、事前学習済みモデルを極性分析タスク向けにファインチューニングせよ。検証セット上でファインチューニングされたモデルの正解率を計測せよ。

In [17]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import (
    BertModel,
    BertTokenizer,
    Trainer,
    TrainingArguments,
    PreTrainedModel,
    AutoConfig
)

# Max Pooling を使った BERTベースの分類モデル
class BertForSequenceClassificationMaxPool(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)  
        self.dropout = nn.Dropout(config.hidden_dropout_prob)  
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)  
        self.init_weights() 
    
    def init_weights(self):
        self.classifier.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
        if self.classifier.bias is not None:
            self.classifier.bias.data.zero_()
    
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):

        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        sequence_output = outputs.last_hidden_state

        input_mask_expanded = attention_mask.unsqueeze(-1).expand(sequence_output.size()).float()
        sequence_output = sequence_output * input_mask_expanded
        max_pooled = torch.max(sequence_output, dim=1)[0]

        pooled_output = self.dropout(max_pooled)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return torch.nn.functional.softmax(logits, dim=-1) if loss is None else (loss, logits)


class_label = train_dataset.features["label"]

config = AutoConfig.from_pretrained(
    model_name,
    num_labels=class_label.num_classes,
    label2id={label: id for id, label in enumerate(class_label.names)},
    id2label={id: label for id, label in enumerate(class_label.names)},
)

model = BertForSequenceClassificationMaxPool(config)
model.bert = BertModel.from_pretrained(model_name)

train_args = TrainingArguments(
    output_dir="output_maxpool",  
    evaluation_strategy="epoch", 
    save_strategy="epoch", 
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_dev_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
eval_results = trainer.evaluate()

print(f"MaxPool Model Evaluation results: {eval_results}")


/home/kawata/.local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_4108591/1330441616.py:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.178500,0.233444,0.916284,0.907692,0.930180,0.918799
2,0.126600,0.274312,0.911697,0.891258,0.941441,0.915663
3,0.091300,0.288723,0.918578,0.909890,0.932432,0.921023


MaxPool Model Evaluation results: {'eval_loss': 0.28872302174568176, 'eval_accuracy': 0.9185779816513762, 'eval_precision': 0.9098901098901099, 'eval_recall': 0.9324324324324325, 'eval_f1': 0.9210233592880979, 'eval_runtime': 5.469, 'eval_samples_per_second': 159.444, 'eval_steps_per_second': 5.12, 'epoch': 3.0}
